# M_TRANSFER 调拨单表探索分析

> 数据治理/数仓建模视角 | 何方珠宝进销存体系

---

## 探索目标
1. 了解调拨单表结构（M_TRANSFER主表 + M_TRANSFERITEM明细表）
2. 识别关键业务字段（发货方、收货方、出入库状态等）
3. 评估数据质量（填充率、一致性）
4. 分析调拨业务场景（总仓→门店、门店→门店、云仓调拨等）
5. 为数仓建模提供依据

In [1]:
# -*- coding: utf-8 -*-
"""
M_TRANSFER 表探索分析 - 调拨单事实表评估
数据治理/数仓建模视角
"""

import sys
import os
# 添加项目路径，以便导入config模块
project_path = r'C:\Users\tianhao\PycharmProjects\hefang_dw'
if project_path not in sys.path:
    sys.path.insert(0, project_path)

import oracledb
import pandas as pd
import numpy as np
from datetime import datetime
from config import ORACLE_CONFIG, ORACLE_DSN

print("="*80)
print("M_TRANSFER 表探索分析 - 调拨单事实表评估")
print("="*80)
print(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

M_TRANSFER 表探索分析 - 调拨单事实表评估
分析时间: 2026-01-16 17:18:20



In [2]:
# 1. 连接Oracle数据库并检查表是否存在
print("【步骤1】连接Oracle数据库...")
conn = oracledb.connect(
    user=ORACLE_CONFIG['user'],
    password=ORACLE_CONFIG['password'],
    dsn=ORACLE_DSN
)
print("✓ 连接成功\n")

# 检查M_TRANSFER和M_TRANSFERITEM表是否存在
print("【步骤1.1】检查M_TRANSFER相关表是否存在...")
cursor = conn.cursor()

# 指定架构（schema）
TARGET_SCHEMA = 'BOSNDS3'  # 目标架构
print(f"目标架构: {TARGET_SCHEMA}")

# 在指定架构中查找
print(f"\n在架构 {TARGET_SCHEMA} 中查找M_TRANSFER相关表...")
sql_check_target_schema = f"""
SELECT OWNER, TABLE_NAME
FROM ALL_TABLES
WHERE UPPER(OWNER) = '{TARGET_SCHEMA}'
  AND UPPER(TABLE_NAME) IN ('M_TRANSFER', 'M_TRANSFERITEM')
ORDER BY TABLE_NAME
"""

cursor.execute(sql_check_target_schema)
tables = cursor.fetchall()

if tables:
    print(f"✓ 在架构 {TARGET_SCHEMA} 中找到表：")
    for row in tables:
        owner = row[0]
        table_name = row[1]
        print(f"  - {owner}.{table_name}")
    
    # 确定要使用的表名和schema
    m_transfer_table = None
    m_transferitem_table = None
    
    for row in tables:
        owner = row[0]
        table_name = row[1]
        if table_name.upper() == 'M_TRANSFER':
            m_transfer_table = f"{owner}.{table_name}"
        elif table_name.upper() == 'M_TRANSFERITEM':
            m_transferitem_table = f"{owner}.{table_name}"
    
    print(f"\n将使用表：")
    print(f"  M_TRANSFER: {m_transfer_table}")
    print(f"  M_TRANSFERITEM: {m_transferitem_table}")
    
    # 保存表名供后续使用
    M_TRANSFER_TABLE = m_transfer_table.split('.')[1] if '.' in m_transfer_table else m_transfer_table
    M_TRANSFERITEM_TABLE = m_transferitem_table.split('.')[1] if '.' in m_transferitem_table else m_transferitem_table
    M_TRANSFER_SCHEMA = m_transfer_table.split('.')[0] if '.' in m_transfer_table else None
    M_TRANSFERITEM_SCHEMA = m_transferitem_table.split('.')[0] if '.' in m_transferitem_table else None
else:
    print(f"⚠️ 在架构 {TARGET_SCHEMA} 中未找到M_TRANSFER或M_TRANSFERITEM表")
    # 使用默认值
    M_TRANSFER_TABLE = 'M_TRANSFER'
    M_TRANSFERITEM_TABLE = 'M_TRANSFERITEM'
    M_TRANSFER_SCHEMA = TARGET_SCHEMA
    M_TRANSFERITEM_SCHEMA = TARGET_SCHEMA

print(f"\n最终使用的表配置：")
print(f"  M_TRANSFER: {M_TRANSFER_SCHEMA}.{M_TRANSFER_TABLE}")
print(f"  M_TRANSFERITEM: {M_TRANSFERITEM_SCHEMA}.{M_TRANSFERITEM_TABLE}")

【步骤1】连接Oracle数据库...
✓ 连接成功

【步骤1.1】检查M_TRANSFER相关表是否存在...
目标架构: BOSNDS3

在架构 BOSNDS3 中查找M_TRANSFER相关表...
✓ 在架构 BOSNDS3 中找到表：
  - BOSNDS3.M_TRANSFER
  - BOSNDS3.M_TRANSFERITEM

将使用表：
  M_TRANSFER: BOSNDS3.M_TRANSFER
  M_TRANSFERITEM: BOSNDS3.M_TRANSFERITEM

最终使用的表配置：
  M_TRANSFER: BOSNDS3.M_TRANSFER
  M_TRANSFERITEM: BOSNDS3.M_TRANSFERITEM


In [3]:
# 2. 查看M_TRANSFER表结构
print("\n【步骤2】查看M_TRANSFER表结构...")
print("="*80)

sql_structure = f"""
SELECT 
    COLUMN_NAME,
    DATA_TYPE,
    DATA_LENGTH,
    DATA_PRECISION,
    DATA_SCALE,
    NULLABLE,
    DATA_DEFAULT
FROM ALL_TAB_COLUMNS
WHERE OWNER = '{M_TRANSFER_SCHEMA}'
  AND TABLE_NAME = '{M_TRANSFER_TABLE}'
ORDER BY COLUMN_ID
"""

try:
    cursor.execute(sql_structure)
    columns_info = cursor.fetchall()
    df_structure = pd.DataFrame(columns_info, columns=[
        '字段名', '数据类型', '长度', '精度', '小数位', '可空', '默认值'
    ])
    
    if len(df_structure) > 0:
        print(f"\nM_TRANSFER表字段结构（共{len(df_structure)}个字段）：")
        print(df_structure.to_string(index=False))
    else:
        print(f"\n⚠️ 未找到表 {M_TRANSFER_TABLE} 的字段信息")
except Exception as e:
    print(f"\n❌ 查询表结构失败: {e}")
    df_structure = pd.DataFrame()


【步骤2】查看M_TRANSFER表结构...

M_TRANSFER表字段结构（共143个字段）：
                  字段名     数据类型  长度   精度  小数位 可空    默认值
                   ID   NUMBER  22 10.0  0.0  N   None
         AD_CLIENT_ID   NUMBER  22 10.0  0.0  Y   None
            AD_ORG_ID   NUMBER  22 10.0  0.0  Y   None
              OWNERID   NUMBER  22 10.0  0.0  Y   None
           MODIFIERID   NUMBER  22 10.0  0.0  Y   None
         CREATIONDATE     DATE   7  NaN  NaN  Y   None
         MODIFIEDDATE     DATE   7  NaN  NaN  Y   None
             ISACTIVE     CHAR   1  NaN  NaN  N   'Y' 
                DOCNO VARCHAR2  80  NaN  NaN  N   None
              DOCTYPE     CHAR   3  NaN  NaN  Y   None
            C_ORIG_ID   NUMBER  22 10.0  0.0  Y   None
            C_DEST_ID   NUMBER  22 10.0  0.0  Y   None
          DESCRIPTION VARCHAR2 765  NaN  NaN  Y   None
            IN_STATUS   NUMBER  22  1.0  0.0  Y      1
               STATUS   NUMBER  22  1.0  0.0  Y      1
             AU_STATE     CHAR   1  NaN  NaN  Y   None
             

In [4]:
# 3. 查看M_TRANSFERITEM表结构
print("\n【步骤3】查看M_TRANSFERITEM表结构...")
print("="*80)

sql_structure_item = f"""
SELECT 
    COLUMN_NAME,
    DATA_TYPE,
    DATA_LENGTH,
    DATA_PRECISION,
    DATA_SCALE,
    NULLABLE,
    DATA_DEFAULT
FROM ALL_TAB_COLUMNS
WHERE OWNER = '{M_TRANSFERITEM_SCHEMA}'
  AND TABLE_NAME = '{M_TRANSFERITEM_TABLE}'
ORDER BY COLUMN_ID
"""

try:
    cursor.execute(sql_structure_item)
    columns_info_item = cursor.fetchall()
    df_structure_item = pd.DataFrame(columns_info_item, columns=[
        '字段名', '数据类型', '长度', '精度', '小数位', '可空', '默认值'
    ])
    
    if len(df_structure_item) > 0:
        print(f"\nM_TRANSFERITEM表字段结构（共{len(df_structure_item)}个字段）：")
        print(df_structure_item.to_string(index=False))
    else:
        print(f"\n⚠️ 未找到表 {M_TRANSFERITEM_TABLE} 的字段信息")
except Exception as e:
    print(f"\n❌ 查询表结构失败: {e}")
    df_structure_item = pd.DataFrame()


【步骤3】查看M_TRANSFERITEM表结构...

M_TRANSFERITEM表字段结构（共41个字段）：
                      字段名     数据类型  长度   精度  小数位 可空  默认值
                       ID   NUMBER  22 10.0  0.0  N None
             AD_CLIENT_ID   NUMBER  22 10.0  0.0  Y None
                AD_ORG_ID   NUMBER  22 10.0  0.0  Y None
                  OWNERID   NUMBER  22 10.0  0.0  Y None
               MODIFIERID   NUMBER  22 10.0  0.0  Y None
             CREATIONDATE     DATE   7  NaN  NaN  Y None
             MODIFIEDDATE     DATE   7  NaN  NaN  Y None
                 ISACTIVE     CHAR   1  NaN  NaN  N 'Y' 
                   STATUS   NUMBER  22  1.0  0.0  Y    1
            M_TRANSFER_ID   NUMBER  22 10.0  0.0  N None
                  ORDERNO   NUMBER  22 10.0  0.0  Y None
             M_PRODUCT_ID   NUMBER  22 10.0  0.0  Y None
M_ATTRIBUTESETINSTANCE_ID   NUMBER  22 10.0  0.0  Y None
                   QTYOUT   NUMBER  22 18.0  0.0  Y    0
                    QTYIN   NUMBER  22 18.0  0.0  Y    0
                  QTYDIFF   N

In [5]:
# 4. 数据量统计
print("\n【步骤4】数据量统计...")
print("="*80)

# 构建表名（包含schema前缀）
m_transfer_full = f"{M_TRANSFER_SCHEMA}.{M_TRANSFER_TABLE}"
m_transferitem_full = f"{M_TRANSFERITEM_SCHEMA}.{M_TRANSFERITEM_TABLE}"

sql_count = f"""
SELECT 
    'M_TRANSFER' AS 表名,
    COUNT(*) AS 总记录数,
    COUNT(DISTINCT ID) AS 唯一主键数,
    COUNT(CASE WHEN ISACTIVE = 'Y' THEN 1 END) AS 有效记录数,
    COUNT(CASE WHEN ISACTIVE = 'N' THEN 1 END) AS 无效记录数,
    MIN(CREATIONDATE) AS 最早创建时间,
    MAX(CREATIONDATE) AS 最新创建时间
FROM {m_transfer_full}
UNION ALL
SELECT 
    'M_TRANSFERITEM' AS 表名,
    COUNT(*) AS 总记录数,
    COUNT(DISTINCT ID) AS 唯一主键数,
    COUNT(CASE WHEN ISACTIVE = 'Y' THEN 1 END) AS 有效记录数,
    COUNT(CASE WHEN ISACTIVE = 'N' THEN 1 END) AS 无效记录数,
    MIN(CREATIONDATE) AS 最早创建时间,
    MAX(CREATIONDATE) AS 最新创建时间
FROM {m_transferitem_full}
"""

try:
    cursor.execute(sql_count)
    count_info = cursor.fetchall()
    df_count = pd.DataFrame(count_info, columns=[
        '表名', '总记录数', '唯一主键数', '有效记录数', '无效记录数', '最早创建时间', '最新创建时间'
    ])
    print(df_count.to_string(index=False))
except Exception as e:
    print(f"❌ 查询数据量失败: {e}")


【步骤4】数据量统计...
            表名    总记录数   唯一主键数   有效记录数  无效记录数              最早创建时间              最新创建时间
    M_TRANSFER 1175559 1175559 1168178   7381 2021-12-15 20:27:07 2026-01-16 17:21:20
M_TRANSFERITEM 2771175 2771175 2771175      0 2021-12-15 20:27:07 2026-01-16 17:21:20


In [6]:
# 5. 查看M_TRANSFER表数据样本（最近10条有效记录）
print("\n【步骤5】M_TRANSFER表数据样本（前10条有效记录）...")
print("="*80)

# 选择关键字段查看样本
sql_sample = f"""
SELECT *
FROM (
    SELECT 
        ID,
        DOCNO,
        BILLDATE,
        TRANSFERTYPE,
        C_ORIG_ID,
        C_DEST_ID,
        TOT_QTY,
        TOT_QTYOUT,
        TOT_QTYIN,
        STATUS,
        CONFIRM_STATUS,
        OUT_STATUS,
        IN_STATUS,
        DATEOUT,
        DATEIN,
        ISACTIVE,
        CREATIONDATE
    FROM {m_transfer_full}
    WHERE ISACTIVE = 'Y'
    ORDER BY CREATIONDATE DESC NULLS LAST
)
WHERE ROWNUM <= 10
"""

try:
    cursor.execute(sql_sample)
    columns = [col[0] for col in cursor.description]
    data = cursor.fetchall()
    df_sample = pd.DataFrame(data, columns=columns)
    
    print(f"\n样本数据（共{len(df_sample)}条）：")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    print(df_sample)
except Exception as e:
    print(f"❌ 查询失败: {e}")


【步骤5】M_TRANSFER表数据样本（前10条有效记录）...

样本数据（共10条）：
         ID            DOCNO  BILLDATE TRANSFERTYPE  C_ORIG_ID  C_DEST_ID  \
0  67176556  TF2601160063917  20260116          NOR        194        101   
1  67176548  TF2601160063909  20260116          NOR        194        105   
2  67176523  TF2601160063884  20260116          NOR        194        101   
3  67176460  TF2601160063821  20260116          NOR        423        101   
4  67176425  TF2601160063786  20260116          NOR        194        104   
5  67176396  TF2601160063757  20260116          NOR        105         99   
6  67176395  TF2601160063756  20260116          NOR        105        101   
7  67176394  TF2601160063755  20260116          NOR        105        101   
8  67176393  TF2601160063754  20260116          NOR        194        101   
9  67176392  TF2601160063753  20260116          NOR        105        101   

   TOT_QTY  TOT_QTYOUT  TOT_QTYIN  STATUS  CONFIRM_STATUS  OUT_STATUS  \
0        4           4         

In [7]:
# 6. 调拨类型分布分析
print("\n【步骤6】调拨类型分布分析...")
print("="*80)

sql_transfer_type = f"""
SELECT 
    TRANSFERTYPE AS 调拨类型,
    CASE TRANSFERTYPE
        WHEN 'ORD' THEN '订单调拨'
        WHEN 'NOR' THEN '正常调拨'
        WHEN 'ADD' THEN '补货调拨'
        WHEN 'DIF' THEN '差异调拨'
        WHEN 'OUT' THEN '代销出库'
        WHEN 'IN' THEN '代销退货'
        WHEN 'EOR' THEN '云仓调拨'
        ELSE '其他'
    END AS 类型说明,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比,
    SUM(TOT_QTY) AS 总调拨数量,
    SUM(TOT_QTYOUT) AS 总出库数量,
    SUM(TOT_QTYIN) AS 总入库数量
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
GROUP BY TRANSFERTYPE
ORDER BY 数量 DESC
"""

try:
    cursor.execute(sql_transfer_type)
    transfer_type_data = cursor.fetchall()
    df_transfer_type = pd.DataFrame(transfer_type_data, columns=[
        '调拨类型', '类型说明', '数量', '占比(%)', '总调拨数量', '总出库数量', '总入库数量'
    ])
    print("\n调拨类型分布：")
    print(df_transfer_type.to_string(index=False))
    
    print("\n📝 调拨类型说明：")
    print("  - ORD(订单调拨): 基于补货订单的调拨")
    print("  - NOR(正常调拨): 常规调拨")
    print("  - ADD(补货调拨): 门店补货")
    print("  - DIF(差异调拨): 差异处理")
    print("  - EOR(云仓调拨): 云仓相关调拨 ⭐关键")
    print("  - OUT(代销出库)/IN(代销退货): 代销业务")
except Exception as e:
    print(f"❌ 查询失败: {e}")


【步骤6】调拨类型分布分析...

调拨类型分布：
调拨类型 类型说明      数量  占比(%)    总调拨数量    总出库数量    总入库数量
 NOR 正常调拨 1159702  99.27  4691563  4683573  4665909
 ORD 订单调拨    7628   0.65 13717795 13714073 13710055
 DIF 差异调拨     847   0.07    20547    20540    34216
 EOR 云仓调拨       1   0.00        1        1        1

📝 调拨类型说明：
  - ORD(订单调拨): 基于补货订单的调拨
  - NOR(正常调拨): 常规调拨
  - ADD(补货调拨): 门店补货
  - DIF(差异调拨): 差异处理
  - EOR(云仓调拨): 云仓相关调拨 ⭐关键
  - OUT(代销出库)/IN(代销退货): 代销业务


In [8]:
# 7. 调拨状态分析
print("\n【步骤7】调拨状态字段分析...")
print("="*80)

# 7.1 提交状态(STATUS)分布
print("\n7.1 提交状态(STATUS)分布：")
sql_status = f"""
SELECT 
    STATUS AS 提交状态,
    CASE STATUS
        WHEN 1 THEN '未提交'
        WHEN 2 THEN '已提交'
        WHEN 3 THEN '待批'
        ELSE '其他'
    END AS 状态说明,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
GROUP BY STATUS
ORDER BY STATUS
"""

try:
    cursor.execute(sql_status)
    status_data = cursor.fetchall()
    df_status = pd.DataFrame(status_data, columns=['提交状态', '状态说明', '数量', '占比(%)'])
    print(df_status.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 7.2 确认状态(CONFIRM_STATUS)分布
print("\n7.2 确认状态(CONFIRM_STATUS)分布：")
sql_confirm = f"""
SELECT 
    CONFIRM_STATUS AS 确认状态,
    CASE CONFIRM_STATUS
        WHEN 1 THEN '未确认'
        WHEN 2 THEN '已确认'
        ELSE '其他'
    END AS 状态说明,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
GROUP BY CONFIRM_STATUS
ORDER BY CONFIRM_STATUS
"""

try:
    cursor.execute(sql_confirm)
    confirm_data = cursor.fetchall()
    df_confirm = pd.DataFrame(confirm_data, columns=['确认状态', '状态说明', '数量', '占比(%)'])
    print(df_confirm.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 7.3 出库状态(OUT_STATUS)分布
print("\n7.3 出库状态(OUT_STATUS)分布：")
sql_out_status = f"""
SELECT 
    OUT_STATUS AS 出库状态,
    CASE OUT_STATUS
        WHEN 1 THEN '未出库'
        WHEN 2 THEN '已出库'
        ELSE '其他'
    END AS 状态说明,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
GROUP BY OUT_STATUS
ORDER BY OUT_STATUS
"""

try:
    cursor.execute(sql_out_status)
    out_status_data = cursor.fetchall()
    df_out_status = pd.DataFrame(out_status_data, columns=['出库状态', '状态说明', '数量', '占比(%)'])
    print(df_out_status.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 7.4 入库状态(IN_STATUS)分布
print("\n7.4 入库状态(IN_STATUS)分布：")
sql_in_status = f"""
SELECT 
    IN_STATUS AS 入库状态,
    CASE IN_STATUS
        WHEN 1 THEN '未入库'
        WHEN 2 THEN '已入库'
        ELSE '其他'
    END AS 状态说明,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
GROUP BY IN_STATUS
ORDER BY IN_STATUS
"""

try:
    cursor.execute(sql_in_status)
    in_status_data = cursor.fetchall()
    df_in_status = pd.DataFrame(in_status_data, columns=['入库状态', '状态说明', '数量', '占比(%)'])
    print(df_in_status.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")


【步骤7】调拨状态字段分析...

7.1 提交状态(STATUS)分布：
 提交状态 状态说明      数量  占比(%)
    1  未提交      81   0.01
    2  已提交 1168097  99.99

7.2 确认状态(CONFIRM_STATUS)分布：
 确认状态 状态说明      数量  占比(%)
    1  未确认      97   0.01
    2  已确认 1168081  99.99

7.3 出库状态(OUT_STATUS)分布：
 出库状态 状态说明      数量  占比(%)
    1  未出库      97   0.01
    2  已出库 1168081  99.99

7.4 入库状态(IN_STATUS)分布：
 入库状态 状态说明      数量  占比(%)
    1  未入库     608   0.05
    2  已入库 1167570  99.95


In [9]:
# 8. 发货方/收货方店仓分析
print("\n【步骤8】发货方/收货方店仓分析...")
print("="*80)

# 8.1 发货方TOP10
print("\n8.1 发货方店仓TOP10（按调拨单数）：")
sql_orig_top = f"""
SELECT *
FROM (
    SELECT 
        s.CODE AS 发货方编码,
        s.NAME AS 发货方名称,
        COUNT(t.ID) AS 调拨单数,
        SUM(t.TOT_QTY) AS 总调拨数量,
        SUM(t.TOT_QTYOUT) AS 总出库数量
    FROM {m_transfer_full} t
    LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE s ON t.C_ORIG_ID = s.ID
    WHERE t.ISACTIVE = 'Y'
    GROUP BY s.CODE, s.NAME
    ORDER BY 调拨单数 DESC
)
WHERE ROWNUM <= 10
"""

try:
    cursor.execute(sql_orig_top)
    orig_data = cursor.fetchall()
    df_orig = pd.DataFrame(orig_data, columns=['发货方编码', '发货方名称', '调拨单数', '总调拨数量', '总出库数量'])
    print(df_orig.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 8.2 收货方TOP10
print("\n8.2 收货方店仓TOP10（按调拨单数）：")
sql_dest_top = f"""
SELECT *
FROM (
    SELECT 
        s.CODE AS 收货方编码,
        s.NAME AS 收货方名称,
        COUNT(t.ID) AS 调拨单数,
        SUM(t.TOT_QTY) AS 总调拨数量,
        SUM(t.TOT_QTYIN) AS 总入库数量
    FROM {m_transfer_full} t
    LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE s ON t.C_DEST_ID = s.ID
    WHERE t.ISACTIVE = 'Y'
    GROUP BY s.CODE, s.NAME
    ORDER BY 调拨单数 DESC
)
WHERE ROWNUM <= 10
"""

try:
    cursor.execute(sql_dest_top)
    dest_data = cursor.fetchall()
    df_dest = pd.DataFrame(dest_data, columns=['收货方编码', '收货方名称', '调拨单数', '总调拨数量', '总入库数量'])
    print(df_dest.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 8.3 调拨流向分析（总仓→门店、门店→门店等）
print("\n8.3 调拨流向分析（发货方类型 → 收货方类型）：")
sql_flow = f"""
SELECT 
    CASE 
        WHEN orig.CODE = '001' THEN '总仓(001)'
        WHEN orig.CODE LIKE 'DS%' THEN '电商(DS%)'
        WHEN orig.CODE LIKE 'RT%' THEN '门店(RT%)'
        ELSE '其他'
    END AS 发货方类型,
    CASE 
        WHEN dest.CODE = '001' THEN '总仓(001)'
        WHEN dest.CODE LIKE 'DS%' THEN '电商(DS%)'
        WHEN dest.CODE LIKE 'RT%' THEN '门店(RT%)'
        ELSE '其他'
    END AS 收货方类型,
    COUNT(t.ID) AS 调拨单数,
    SUM(t.TOT_QTY) AS 总调拨数量,
    ROUND(COUNT(t.ID) * 100.0 / SUM(COUNT(t.ID)) OVER (), 2) AS 占比
FROM {m_transfer_full} t
LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE orig ON t.C_ORIG_ID = orig.ID
LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE dest ON t.C_DEST_ID = dest.ID
WHERE t.ISACTIVE = 'Y'
GROUP BY 
    CASE 
        WHEN orig.CODE = '001' THEN '总仓(001)'
        WHEN orig.CODE LIKE 'DS%' THEN '电商(DS%)'
        WHEN orig.CODE LIKE 'RT%' THEN '门店(RT%)'
        ELSE '其他'
    END,
    CASE 
        WHEN dest.CODE = '001' THEN '总仓(001)'
        WHEN dest.CODE LIKE 'DS%' THEN '电商(DS%)'
        WHEN dest.CODE LIKE 'RT%' THEN '门店(RT%)'
        ELSE '其他'
    END
ORDER BY 调拨单数 DESC
"""

try:
    cursor.execute(sql_flow)
    flow_data = cursor.fetchall()
    df_flow = pd.DataFrame(flow_data, columns=['发货方类型', '收货方类型', '调拨单数', '总调拨数量', '占比(%)'])
    print(df_flow.to_string(index=False))
    
    print("\n📝 调拨流向说明：")
    print("  - 总仓→门店: 总仓发货至线下门店（主要补货场景）")
    print("  - 门店→门店: 门店间调拨（店间调货）")
    print("  - 门店→总仓: 门店退货至总仓")
    print("  - 总仓→电商: 电商发货")
except Exception as e:
    print(f"  查询失败: {e}")


【步骤8】发货方/收货方店仓分析...

8.1 发货方店仓TOP10（按调拨单数）：
发货方编码                      发货方名称   调拨单数    总调拨数量    总出库数量
  001                       中山总仓 751237  1989388  1988611
DS001         HEFANG JEWELRY(天猫) 166835   214098   214030
  010                      配件辅料仓  75848 14896097 14887936
DS009         HEFANG JEWELRY(抖音)  46727    59104    59082
DS002         HEFANG JEWELRY(京东)  13181    16474    16467
  005                        线下仓  12011   268165   267999
DS006 HEFANG JEWELRY何方珠宝旗舰店(小红书)  11554    16414    16334
DS010        HEFANG JEWELRY(唯品会)   9723    17657    17639
  016                        售后仓   8913   356611   356262
  015                        唯品仓   8116    29038    29002

8.2 收货方店仓TOP10（按调拨单数）：
收货方编码                      收货方名称   调拨单数   总调拨数量   总入库数量
DS001         HEFANG JEWELRY(天猫) 532857 1095369 1095216
  016                        售后仓 261884  351272  350986
DS009         HEFANG JEWELRY(抖音) 108094  226356  226268
DS002         HEFANG JEWELRY(京东)  46034   89589   89589
DS006 HEFANG 

In [10]:
# 9. 日期字段分析
print("\n【步骤9】日期字段分析...")
print("="*80)

# 9.1 BILLDATE（单据日期）填充率
print("\n9.1 BILLDATE（单据日期）字段分析：")
sql_billdate = f"""
SELECT 
    COUNT(*) AS 总记录数,
    COUNT(BILLDATE) AS 有单据日期的记录数,
    COUNT(*) - COUNT(BILLDATE) AS 无单据日期的记录数,
    ROUND(COUNT(BILLDATE) * 100.0 / COUNT(*), 2) AS 填充率,
    MIN(BILLDATE) AS 最早单据日期,
    MAX(BILLDATE) AS 最新单据日期
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_billdate)
    billdate_info = cursor.fetchone()
    if billdate_info:
        print(f"  总记录数: {billdate_info[0]}")
        print(f"  有单据日期的记录数: {billdate_info[1]}")
        print(f"  填充率: {billdate_info[3]}%")
        print(f"  最早单据日期: {billdate_info[4]}")
        print(f"  最新单据日期: {billdate_info[5]}")
except Exception as e:
    print(f"  查询失败: {e}")

# 9.2 DATEOUT（出库日期）填充率
print("\n9.2 DATEOUT（出库日期）字段分析：")
sql_dateout = f"""
SELECT 
    COUNT(*) AS 总记录数,
    COUNT(DATEOUT) AS 有出库日期的记录数,
    ROUND(COUNT(DATEOUT) * 100.0 / COUNT(*), 2) AS 填充率,
    MIN(DATEOUT) AS 最早出库日期,
    MAX(DATEOUT) AS 最新出库日期
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_dateout)
    dateout_info = cursor.fetchone()
    if dateout_info:
        print(f"  总记录数: {dateout_info[0]}")
        print(f"  有出库日期的记录数: {dateout_info[1]}")
        print(f"  填充率: {dateout_info[2]}%")
        print(f"  最早出库日期: {dateout_info[3]}")
        print(f"  最新出库日期: {dateout_info[4]}")
except Exception as e:
    print(f"  查询失败: {e}")

# 9.3 DATEIN（入库日期）填充率
print("\n9.3 DATEIN（入库日期）字段分析：")
sql_datein = f"""
SELECT 
    COUNT(*) AS 总记录数,
    COUNT(DATEIN) AS 有入库日期的记录数,
    ROUND(COUNT(DATEIN) * 100.0 / COUNT(*), 2) AS 填充率,
    MIN(DATEIN) AS 最早入库日期,
    MAX(DATEIN) AS 最新入库日期
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_datein)
    datein_info = cursor.fetchone()
    if datein_info:
        print(f"  总记录数: {datein_info[0]}")
        print(f"  有入库日期的记录数: {datein_info[1]}")
        print(f"  填充率: {datein_info[2]}%")
        print(f"  最早入库日期: {datein_info[3]}")
        print(f"  最新入库日期: {datein_info[4]}")
except Exception as e:
    print(f"  查询失败: {e}")


【步骤9】日期字段分析...

9.1 BILLDATE（单据日期）字段分析：
  总记录数: 1168178
  有单据日期的记录数: 1168178
  填充率: 100%
  最早单据日期: 20211130
  最新单据日期: 20260116

9.2 DATEOUT（出库日期）字段分析：
  总记录数: 1168178
  有出库日期的记录数: 1168081
  填充率: 99.99%
  最早出库日期: 20211215
  最新出库日期: 20260116

9.3 DATEIN（入库日期）字段分析：
  总记录数: 1168178
  有入库日期的记录数: 1167570
  填充率: 99.95%
  最早入库日期: 20211217
  最新入库日期: 20260116


In [11]:
# 10. 数量字段一致性分析
print("\n【步骤10】数量字段一致性分析...")
print("="*80)

# 10.1 调拨数量 vs 出库数量 vs 入库数量 对比
print("\n10.1 调拨数量 vs 出库数量 vs 入库数量 对比：")
sql_qty_compare = f"""
SELECT 
    COUNT(*) AS 总记录数,
    SUM(TOT_QTY) AS 总调拨数量,
    SUM(TOT_QTYOUT) AS 总出库数量,
    SUM(TOT_QTYIN) AS 总入库数量,
    COUNT(CASE WHEN TOT_QTY = TOT_QTYOUT AND TOT_QTYOUT = TOT_QTYIN THEN 1 END) AS 数量完全一致,
    COUNT(CASE WHEN TOT_QTY > 0 AND TOT_QTYOUT = 0 THEN 1 END) AS 未出库,
    COUNT(CASE WHEN TOT_QTYOUT > 0 AND TOT_QTYIN = 0 THEN 1 END) AS 已出库未入库,
    COUNT(CASE WHEN TOT_QTYIN > 0 AND TOT_QTYIN < TOT_QTYOUT THEN 1 END) AS 部分入库,
    COUNT(CASE WHEN TOT_QTYIN <> TOT_QTYOUT AND TOT_QTYIN > 0 AND TOT_QTYOUT > 0 THEN 1 END) AS 出入库数量不一致
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_qty_compare)
    qty_info = cursor.fetchone()
    if qty_info:
        total = qty_info[0]
        print(f"  总记录数: {total:,}")
        print(f"  总调拨数量: {qty_info[1]:,}")
        print(f"  总出库数量: {qty_info[2]:,}")
        print(f"  总入库数量: {qty_info[3]:,}")
        print(f"  数量完全一致: {qty_info[4]:,} ({qty_info[4]*100.0/total:.2f}%)")
        print(f"  未出库: {qty_info[5]:,} ({qty_info[5]*100.0/total:.2f}%)")
        print(f"  已出库未入库: {qty_info[6]:,} ({qty_info[6]*100.0/total:.2f}%)")
        print(f"  部分入库: {qty_info[7]:,}")
        print(f"  出入库数量不一致: {qty_info[8]:,}")
except Exception as e:
    print(f"  查询失败: {e}")

# 10.2 主表与明细表数量一致性
print("\n10.2 主表与明细表数量一致性：")
sql_consistency = f"""
SELECT 
    COUNT(DISTINCT mt.ID) AS 调拨单数,
    COUNT(CASE WHEN ABS(mt.TOT_QTY - COALESCE(item_sum.TOT_QTY, 0)) <= 0.01 THEN 1 END) AS 数量一致的单数,
    COUNT(CASE WHEN ABS(mt.TOT_QTY - COALESCE(item_sum.TOT_QTY, 0)) > 0.01 THEN 1 END) AS 数量不一致的单数,
    ROUND(COUNT(CASE WHEN ABS(mt.TOT_QTY - COALESCE(item_sum.TOT_QTY, 0)) <= 0.01 THEN 1 END) * 100.0 / COUNT(DISTINCT mt.ID), 2) AS 一致性比例
FROM {m_transfer_full} mt
LEFT JOIN (
    SELECT 
        M_TRANSFER_ID,
        SUM(QTY) AS TOT_QTY
    FROM {m_transferitem_full}
    WHERE ISACTIVE = 'Y'
    GROUP BY M_TRANSFER_ID
) item_sum ON mt.ID = item_sum.M_TRANSFER_ID
WHERE mt.ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_consistency)
    consistency_info = cursor.fetchone()
    if consistency_info:
        print(f"  调拨单数: {consistency_info[0]:,}")
        print(f"  数量一致的单数: {consistency_info[1]:,}")
        print(f"  数量不一致的单数: {consistency_info[2]:,}")
        print(f"  一致性比例: {consistency_info[3]:.2f}%")
except Exception as e:
    print(f"  查询失败: {e}")


【步骤10】数量字段一致性分析...

10.1 调拨数量 vs 出库数量 vs 入库数量 对比：
  总记录数: 1,168,178
  总调拨数量: 18,429,906
  总出库数量: 18,418,187
  总入库数量: 18,410,181
  数量完全一致: 1,165,159 (99.74%)
  未出库: 261 (0.02%)
  已出库未入库: 914 (0.08%)
  部分入库: 469
  出入库数量不一致: 470

10.2 主表与明细表数量一致性：
  调拨单数: 1,168,178
  数量一致的单数: 1,168,174
  数量不一致的单数: 0
  一致性比例: 100.00%


In [12]:
# 11. 月度调拨趋势分析
print("\n【步骤11】月度调拨趋势分析...")
print("="*80)

sql_monthly = f"""
SELECT *
FROM (
    SELECT 
        TO_CHAR(TO_DATE(TO_CHAR(BILLDATE), 'YYYYMMDD'), 'YYYY-MM') AS 年月,
        COUNT(*) AS 调拨单数,
        SUM(TOT_QTY) AS 调拨数量,
        SUM(TOT_QTYOUT) AS 出库数量,
        SUM(TOT_QTYIN) AS 入库数量,
        ROUND(SUM(TOT_AMTQTY_LIST), 2) AS 调拨金额
    FROM {m_transfer_full}
    WHERE ISACTIVE = 'Y'
      AND BILLDATE IS NOT NULL
      AND BILLDATE > 0
    GROUP BY TO_CHAR(TO_DATE(TO_CHAR(BILLDATE), 'YYYYMMDD'), 'YYYY-MM')
    ORDER BY 年月 DESC
)
WHERE ROWNUM <= 12
"""

try:
    cursor.execute(sql_monthly)
    monthly_data = cursor.fetchall()
    if monthly_data:
        df_monthly = pd.DataFrame(monthly_data, columns=[
            '年月', '调拨单数', '调拨数量', '出库数量', '入库数量', '调拨金额'
        ])
        print("\n最近12个月调拨趋势：")
        print(df_monthly.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")


【步骤11】月度调拨趋势分析...

最近12个月调拨趋势：
     年月  调拨单数    调拨数量    出库数量    入库数量         调拨金额
2026-01 23735  126567  126027  120400  51981589.50
2025-12 41142  429100  428710  428428 100190100.60
2025-11 47107  617143  616701  616234 119251013.90
2025-10 56097  218055  217678  217474 101752128.00
2025-09 36178  212175  212124  211644  81393562.24
2025-08 40112  214963  214912  215311  76627381.77
2025-07 28580 1170758 1170443 1169682 105044994.15
2025-06 44171  648974  648703  648599 137115545.36
2025-05 49468  557162  556813  556723 109203168.49
2025-04 26407 1078290 1078092 1078064 131104940.01
2025-03 27940  109752  109676  109673  86544100.26
2025-02 23428  659051  657243  657237  70921151.96


In [13]:
# 12. 云仓调拨专题分析
print("\n【步骤12】云仓调拨专题分析...")
print("="*80)

# 12.1 云仓调拨（TRANSFERTYPE='EOR'）统计
print("\n12.1 云仓调拨（TRANSFERTYPE='EOR'）统计：")
sql_eor = f"""
SELECT 
    COUNT(*) AS 云仓调拨单数,
    SUM(TOT_QTY) AS 总调拨数量,
    SUM(TOT_QTYOUT) AS 总出库数量,
    SUM(TOT_QTYIN) AS 总入库数量,
    ROUND(SUM(TOT_AMTQTY_LIST), 2) AS 总调拨金额
FROM {m_transfer_full}
WHERE ISACTIVE = 'Y'
  AND TRANSFERTYPE = 'EOR'
"""

try:
    cursor.execute(sql_eor)
    eor_info = cursor.fetchone()
    if eor_info:
        print(f"  云仓调拨单数: {eor_info[0]:,}")
        print(f"  总调拨数量: {eor_info[1]:,}" if eor_info[1] else "  总调拨数量: 0")
        print(f"  总出库数量: {eor_info[2]:,}" if eor_info[2] else "  总出库数量: 0")
        print(f"  总入库数量: {eor_info[3]:,}" if eor_info[3] else "  总入库数量: 0")
        print(f"  总调拨金额: {eor_info[4]:,.2f}" if eor_info[4] else "  总调拨金额: 0")
except Exception as e:
    print(f"  查询失败: {e}")

# 12.2 涉及云仓门店的调拨（发货方或收货方是云仓）
print("\n12.2 涉及云仓门店的调拨（IS_ALLO2OSTORAGE='Y'的店仓）：")
sql_cloud_store = f"""
SELECT 
    CASE 
        WHEN orig.IS_ALLO2OSTORAGE = 'Y' AND dest.IS_ALLO2OSTORAGE = 'Y' THEN '云仓→云仓'
        WHEN orig.IS_ALLO2OSTORAGE = 'Y' THEN '云仓→非云仓'
        WHEN dest.IS_ALLO2OSTORAGE = 'Y' THEN '非云仓→云仓'
        ELSE '非云仓→非云仓'
    END AS 调拨方向,
    COUNT(t.ID) AS 调拨单数,
    SUM(t.TOT_QTY) AS 总调拨数量,
    ROUND(COUNT(t.ID) * 100.0 / SUM(COUNT(t.ID)) OVER (), 2) AS 占比
FROM {m_transfer_full} t
LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE orig ON t.C_ORIG_ID = orig.ID
LEFT JOIN {M_TRANSFER_SCHEMA}.C_STORE dest ON t.C_DEST_ID = dest.ID
WHERE t.ISACTIVE = 'Y'
GROUP BY 
    CASE 
        WHEN orig.IS_ALLO2OSTORAGE = 'Y' AND dest.IS_ALLO2OSTORAGE = 'Y' THEN '云仓→云仓'
        WHEN orig.IS_ALLO2OSTORAGE = 'Y' THEN '云仓→非云仓'
        WHEN dest.IS_ALLO2OSTORAGE = 'Y' THEN '非云仓→云仓'
        ELSE '非云仓→非云仓'
    END
ORDER BY 调拨单数 DESC
"""

try:
    cursor.execute(sql_cloud_store)
    cloud_data = cursor.fetchall()
    df_cloud = pd.DataFrame(cloud_data, columns=['调拨方向', '调拨单数', '总调拨数量', '占比(%)'])
    print(df_cloud.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")


【步骤12】云仓调拨专题分析...

12.1 云仓调拨（TRANSFERTYPE='EOR'）统计：
  云仓调拨单数: 1
  总调拨数量: 1
  总出库数量: 1
  总入库数量: 1
  总调拨金额: 20.00

12.2 涉及云仓门店的调拨（IS_ALLO2OSTORAGE='Y'的店仓）：
   调拨方向    调拨单数    总调拨数量  占比(%)
非云仓→非云仓 1112236 16807580  95.21
 非云仓→云仓   26293  1527771   2.25
 云仓→非云仓   23298    81005   1.99
  云仓→云仓    6351    13550   0.54


In [14]:
# 13. 综合评估 - M_TRANSFER作为调拨事实表的可行性
print("\n【步骤13】综合评估 - M_TRANSFER作为调拨事实表的可行性...")
print("="*80)
print("\n" + "="*80)
print("📊 M_TRANSFER表作为「调拨事实表」的评估报告")
print("="*80)

assessments = []

# 1. 表结构评估
assessments.append("✅ 表结构：主表(M_TRANSFER) + 明细表(M_TRANSFERITEM) 结构合理")

# 2. 关键维度字段
try:
    cursor.execute(f"""
        SELECT 
            COUNT(*) AS total,
            COUNT(C_ORIG_ID) AS orig_count,
            COUNT(C_DEST_ID) AS dest_count,
            COUNT(BILLDATE) AS billdate_count
        FROM {m_transfer_full}
        WHERE ISACTIVE = 'Y'
    """)
    dim_info = cursor.fetchone()
    if dim_info:
        total = dim_info[0]
        orig_rate = dim_info[1] * 100.0 / total
        dest_rate = dim_info[2] * 100.0 / total
        date_rate = dim_info[3] * 100.0 / total
        if orig_rate >= 95 and dest_rate >= 95:
            assessments.append(f"✅ 维度字段：发货方{orig_rate:.1f}%、收货方{dest_rate:.1f}%填充率良好")
        else:
            assessments.append(f"⚠️ 维度字段：发货方{orig_rate:.1f}%、收货方{dest_rate:.1f}%填充率")
except:
    assessments.append("⚠️ 维度字段：无法评估")

# 3. 状态字段完整性
try:
    cursor.execute(f"""
        SELECT 
            COUNT(CASE WHEN OUT_STATUS = 2 THEN 1 END) AS out_done,
            COUNT(CASE WHEN IN_STATUS = 2 THEN 1 END) AS in_done,
            COUNT(*) AS total
        FROM {m_transfer_full}
        WHERE ISACTIVE = 'Y'
    """)
    status_info = cursor.fetchone()
    if status_info:
        total = status_info[2]
        out_rate = status_info[0] * 100.0 / total
        in_rate = status_info[1] * 100.0 / total
        assessments.append(f"✅ 状态字段：已出库{out_rate:.1f}%、已入库{in_rate:.1f}%")
except:
    assessments.append("⚠️ 状态字段：无法评估")

# 4. 数量字段
try:
    cursor.execute(f"""
        SELECT 
            SUM(TOT_QTY),
            SUM(TOT_QTYOUT),
            SUM(TOT_QTYIN)
        FROM {m_transfer_full}
        WHERE ISACTIVE = 'Y'
    """)
    qty_info = cursor.fetchone()
    if qty_info and qty_info[0]:
        assessments.append(f"✅ 数量字段：总调拨{qty_info[0]:,}件、出库{qty_info[1]:,}件、入库{qty_info[2]:,}件")
except:
    assessments.append("⚠️ 数量字段：无法评估")

# 5. 调拨类型丰富度
try:
    cursor.execute(f"""
        SELECT COUNT(DISTINCT TRANSFERTYPE)
        FROM {m_transfer_full}
        WHERE ISACTIVE = 'Y'
    """)
    type_count = cursor.fetchone()[0]
    assessments.append(f"✅ 调拨类型：发现{type_count}种调拨类型（NOR/ORD/ADD/EOR等）")
except:
    assessments.append("⚠️ 调拨类型：无法评估")

print("\n评估结果：")
for i, assessment in enumerate(assessments, 1):
    print(f"{i}. {assessment}")

print("\n" + "="*80)
print("💡 结论与建议：")
print("="*80)
print("1. ✅ M_TRANSFER表包含完整的调拨业务信息（发货方、收货方、出入库状态）")
print("2. ✅ 主表与明细表结构清晰，支持不同粒度的分析")
print("3. ✅ 关键字段（C_ORIG_ID、C_DEST_ID、状态、数量）填充率较高")
print("4. ⚠️ 建议：")
print("   - 使用 OUT_STATUS=2 AND IN_STATUS=2 筛选已完成调拨")
print("   - 使用 BILLDATE 作为业务日期维度")
print("   - 使用 TRANSFERTYPE 区分调拨类型（尤其关注EOR云仓调拨）")
print("   - 结合 C_STORE 表分析调拨流向（总仓→门店、门店→门店等）")
print("   - 明细表 M_TRANSFERITEM 可用于SKU级别的调拨分析")
print("="*80)


【步骤13】综合评估 - M_TRANSFER作为调拨事实表的可行性...

📊 M_TRANSFER表作为「调拨事实表」的评估报告

评估结果：
1. ✅ 表结构：主表(M_TRANSFER) + 明细表(M_TRANSFERITEM) 结构合理
2. ✅ 维度字段：发货方100.0%、收货方100.0%填充率良好
3. ✅ 状态字段：已出库100.0%、已入库99.9%
4. ✅ 数量字段：总调拨18,429,906件、出库18,418,187件、入库18,410,181件
5. ✅ 调拨类型：发现4种调拨类型（NOR/ORD/ADD/EOR等）

💡 结论与建议：
1. ✅ M_TRANSFER表包含完整的调拨业务信息（发货方、收货方、出入库状态）
2. ✅ 主表与明细表结构清晰，支持不同粒度的分析
3. ✅ 关键字段（C_ORIG_ID、C_DEST_ID、状态、数量）填充率较高
4. ⚠️ 建议：
   - 使用 OUT_STATUS=2 AND IN_STATUS=2 筛选已完成调拨
   - 使用 BILLDATE 作为业务日期维度
   - 使用 TRANSFERTYPE 区分调拨类型（尤其关注EOR云仓调拨）
   - 结合 C_STORE 表分析调拨流向（总仓→门店、门店→门店等）
   - 明细表 M_TRANSFERITEM 可用于SKU级别的调拨分析


In [15]:
# 14. 关闭数据库连接
print("\n【步骤14】关闭数据库连接...")
cursor.close()
conn.close()
print("✓ 连接已关闭")

print("\n" + "="*80)
print("探索分析完成！")
print("="*80)


【步骤14】关闭数据库连接...
✓ 连接已关闭

探索分析完成！


---

## 📋 M_TRANSFER 表探索总结

### 关键字段速查

| 字段 | 说明 | 重要性 |
|------|------|--------|
| **DOCNO** | 单据编号 | 业务键 |
| **BILLDATE** | 单据日期（NUMBER, YYYYMMDD） | ⭐时间维度 |
| **TRANSFERTYPE** | 调拨类型（NOR/ORD/ADD/EOR等） | ⭐业务分类 |
| **C_ORIG_ID** | 发货方店仓ID | ⭐发货维度 |
| **C_DEST_ID** | 收货方店仓ID | ⭐收货维度 |
| **TOT_QTY** | 总调拨数量 | 度量 |
| **TOT_QTYOUT** | 总出库数量 | ⭐度量 |
| **TOT_QTYIN** | 总入库数量 | ⭐度量 |
| **STATUS** | 提交状态（1未提交/2提交） | 状态 |
| **OUT_STATUS** | 出库状态（1未出库/2已出库） | ⭐状态 |
| **IN_STATUS** | 入库状态（1未入库/2已入库） | ⭐状态 |
| **DATEOUT** | 出库日期 | 时间维度 |
| **DATEIN** | 入库日期 | 时间维度 |

### 调拨类型说明

| 类型代码 | 说明 |
|----------|------|
| NOR | 正常调拨 |
| ORD | 订单调拨 |
| ADD | 补货调拨 |
| DIF | 差异调拨 |
| **EOR** | **云仓调拨** ⭐ |
| OUT | 代销出库 |
| IN | 代销退货 |

### 数仓建模建议

```sql
-- 筛选已完成调拨
WHERE ISACTIVE = 'Y'
  AND OUT_STATUS = 2  -- 已出库
  AND IN_STATUS = 2   -- 已入库

-- 筛选云仓调拨
WHERE TRANSFERTYPE = 'EOR'

-- 关联店仓获取流向
LEFT JOIN C_STORE orig ON t.C_ORIG_ID = orig.ID
LEFT JOIN C_STORE dest ON t.C_DEST_ID = dest.ID
```